# National Stock Exchange Kenya (NSE) Prediction using Linear Regression
This notebook describes the process of predicting stock exchange prices of Diamond Trust Bank company using Linear Regression.

The data is scraped from the official [NSE Kenya Website](https://www.nse.co.ke/market-statistics.html). The scraping is done by a simple python script worker deployed on Heroku. The scraper then stores this data into a Google Spreadsheets document. This notebook thus gets the data from that Spreadsheet for use in the analysis

First, import the necessary modules:

In [8]:
import csv
import math
import pandas as pd
import numpy as np

from sklearn import preprocessing, cross_validation, svm
from sklearn.linear_model import LinearRegression
from spreadsheet import GoogleSpreadSheets

In [9]:
sheets = GoogleSpreadSheets("NSE Stocks", "DTK")

In [10]:
dtk_data = sheets.get_all_records(head=2)

Let's convert the data into csv format for better handling

In [11]:
keys = dtk_data[0].keys()
with open('dtk_stocks.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(dtk_data)

In [12]:
dtk_stocks = pd.read_csv('dtk_stocks.csv')
print dtk_stocks.shape
dtk_stocks.head()

(20, 8)


,company,volume,last_traded_price_ksh,percentage_change,date,prev_price_ksh,low_ksh,high_ksh
0,Diamond Trust Bank Kenya Ltd Ord 4.00,1500.0,190,-0.52,Thu Nov 16 22:58:11 2017,191.0,190.0,190.0
1,Diamond Trust Bank Kenya Ltd Ord 4.00,24300.0,187,-1.58,Fri Nov 17 22:38:53 2017,190.0,186.0,190.0
2,Diamond Trust Bank Kenya Ltd Ord 4.00,32100.0,190,NaN,Mon Nov 20 11:01:00 2017,NaN,NaN,NaN
3,Diamond Trust Bank Kenya Ltd Ord 4.00,32100.0,190,1.60,Tue Nov 21 15:01:15 2017,187.0,186.0,191.0
4,Diamond Trust Bank Kenya Ltd Ord 4.00,400.0,175,NaN,Mon Oct 23 11:01:00 2017,NaN,NaN,NaN


Filter out columns / features that will not be used

In [14]:
features = ['volume', 'last_traded_price_ksh']
dtk_stocks = dtk_stocks[features]
dtk_stocks.head()

,volume,last_traded_price_ksh
0,1500.0,190
1,24300.0,187
2,32100.0,190
3,32100.0,190
4,400.0,175


In [30]:
forecast_feature = 'last_traded_price_ksh' #  The column to be predicted by the model

# Set default values for Undefined data. This will be treated as an outlier
dtk_stocks.fillna(-99999, inplace=True) 

# Try to predict 10 percent of the data frame. 
# Using data from 10% of total days ago to predict today
forecast_out = int(math.ceil(0.1*len(dtk_stocks))) 
print forecast_out

# Make each column the last traded price 10% of total days into the future. Shift it 10% of total days
dtk_stocks['label'] = dtk_stocks[forecast_feature].shift(-forecast_out) 
dtk_stocks.dropna(inplace=True)
dtk_stocks.head()

2


,volume,last_traded_price_ksh,label
0,1500.0,190,190.0
1,24300.0,187,190.0
2,32100.0,190,175.0
3,32100.0,190,175.0
4,400.0,175,178.0


Features will be defined as uppercase X and labels as lowercase y

In [31]:
X = np.array(dtk_stocks.drop(['label'], axis=1)) # Drop the 'label' feature. Returns a new dataframe
y = np.array(dtk_stocks['label'])

Now, scale / normalize the data to fit `-1` to `1` values

In [32]:
X = preprocessing.scale(X)
# X = X[:-forecast_out+1] # Want to make sure we have X's only where we have values for Y

y = np.array(dtk_stocks['label'])
print len(X), len(y) # Check to see if the lengths are equal

16 16
